# Part 4-0: Create a custom container

We will be using [SageMaker Studio Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines.html) to automate and run jobs in our pipleline. SageMaker Pipelines uses purpose built docker containers behind the scene to run jobs (aka [Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html)) in a sequence that you define (much like a DevOps CI/CD pipeline). You can build our own docker container with Python3, [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) and [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) installs, so that you can make use of them to make calls to Amazon Fraud Detector APIs via Boto3 library and access SageMaker constructs such as Feature Store etc. via custom data processing scripts.

To achieve that, you will first have to build a docker image and push it to an [ECR (Elastic Container Registry)](https://aws.amazon.com/ecr/) repo in your account. Typically this can be done using `docker` CLI and `aws cli` in your local machine pretty easily. However, SageMaker makes it even easier to use this studio environment to build, create, and push any custom container to your ECR repository using a purpose built tool known as `sagemaker-studio-image-build` and use the custom container image in your Notebooks for your ML projects. 

For more information on this, refer to [this blog post](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/)

Next, install this required CLI tool into our SageMaker environment.

In [ ]:
import boto3
region = boto3.Session().region_name
MY_ACCOUNT = boto3.client('sts').get_caller_identity().get('Account')
#CONTAINER_IMAGE_URI="738335684114.dkr.ecr.us-east-1.amazonaws.com/nlp-script-processor:1.0"
nlp_script_processor = f"nlp-script-processor:1.0"

CONTAINER_IMAGE_URI=f"{MY_ACCOUNT}.dkr.ecr.{region}.amazonaws.com/{nlp_script_processor}"
CONTAINER_IMAGE_URI

In [ ]:
# Install sagemaker-studio-image-build CLI tool
!pip install sagemaker-studio-image-build

# 1. Grant appropriate permissions to SageMaker
---
In order to be able to use `sagemaker-studio-image-build`, we need to first add permission to SageMaker's IAM role so that it may perform actions on your behalf. Specifically, you would add Amazon ECR and Amazon CodeBuild permissions to it. Add the <font color='green'> AmazonEC2ContainerRegistryFullAccess </font> and <font color='green'> AWSCodeBuildAdminAccess </font> policies to your Sagemaker default role.

<img src="./images/iam-perm.png" width="500" height="600"/>


In addition to this, you will also have to add `iam:PassRole` permission to the SageMaker Studio execution role. Add the Policy document below as an inline policy to the SageMaker Studio Execution role in IAM console.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/*",
            "Condition": {
                "StringLikeIfExists": {
                    "iam:PassedToService": "codebuild.amazonaws.com"
                }
            }
        }
    ]
}
```

As a last and final step, you must also add a trust relationship in the SageMaker Studio Execution role to allow CodeBuild to assume this role. To add a trust relationship
* Navigate to IAM Console
* Search for your SageMaker execution role. (You can find your Sagemaker execution role name from Sagemaker Studio console)
* Click on the "Trust Relationships" tab > Click the "Edit Trust relationship" button
* Add the following Trust relationship to any pre-existing trust relationship

```json
{
  "Effect": "Allow",
  "Principal": {
    "Service": "codebuild.amazonaws.com"
  },
  "Action": "sts:AssumeRole"
}
```

In a normal situation, your final trust relationship should look something like this

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "codebuild.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

<img src="./images/trust.png" width="500" height="600"/>

<div class="alert alert-warning"> <h4 color="red"><strong>💡 NOTE </strong></h4>
IAM Policies described in this notebook can be overly permissive. Please practice caution in setting up IAM Roles with them. For fine grained permissions for the sagemaker-studio-image-build tool, please refer to <a href="https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/">this post</a>. For best practices on SageMaker security, IAM roles and policies refer to this <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/security_iam_id-based-policy-examples.html#security_iam_service-with-iam-policy-best-practices">document</a>.
</div>

# 2. Build a custom Docker image
---

We will now build a create a custom [Dockerfile](https://docs.docker.com/engine/reference/builder/) and use the CLI tool to build the image from the Dockerfile. Our docker image is going to be pretty simple, it will be a copy of the open source [python:3.7-slim-buster](https://github.com/docker-library/python/blob/117d4e375b86cdbe1853930478d0d07d7d5701f7/3.7/buster/slim/Dockerfile) image and contain an installation of [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html), [SageMaker SDK](https://github.com/aws/sagemaker-python-sdk), Pandas, and Numpy.

For our NLP pipeline, we have a number of tasks that depend on Boto3, SageMaker SDK. We will also use the SageFin Client side SDK to be able to download 10k/10Q reports from SEC's EDGAR system. We will install all of these dependencies in the container and use this custom container in our [`ScriptProcessor` step](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html) in our pipelines. We will also install the [`smjsindustry` library](https://sagemaker-jumpstart-industry-pack.readthedocs.io/en/latest/notebooks/finance/notebook1/SEC_Retrieval_Summarizer_Scoring.html) using which we will retrieve the SEC reports, in the container.

In [ ]:
%%writefile Dockerfile
FROM python:3.7-slim-buster

RUN pip3 install smjsindustry==1.0.0 requests botocore boto3>=1.15.0 sagemaker pandas numpy transformers typing sentencepiece nltk
RUN python3 -c "import nltk; nltk.download('punkt')"
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

The code cell above will create a `Dockerfile` in the local project's directory, we can then run the `sm-docker build` command to build, and publish our image. This single command will take care of building the Docker image and publishing it to a [private ECR Repository](https://docs.aws.amazon.com/AmazonECR/latest/userguide/Repositories.html) in your current region (i.e. your SageMaker Studio's default Region). 

<font color='red'> NOTE: </font> You must execute the code cell above in order to be able to run the following cell. `sm-docker build` reads the `Dcokerfile` to create the docker image. To ensure that the code above ran successfully, please verify that you have a file named `Dockerfile` under the project's root directory in the "File Browser" in the left panel of Studio. This project already includes the Dockerfile, however, if you modify the code cell above, it would be a good idea to verify if the contents of the Dockerfile were updated correctly.


In [ ]:
%%time

!sm-docker build . --repository $nlp_script_processor

Running the command in the above code cell will print log lines in the notebook ending with three lines that look like this-

```sh
[Container] 2021/05/15 03:19:43 Phase complete: POST_BUILD State: SUCCEEDED
[Container] 2021/05/15 03:19:43 Phase context status code:  Message:
Image URI: <ACCOUNT_ID>.dkr.ecr.<REGION>.amazonaws.com/sagemaker-studio-d-xxxxxxxxx:default-<xxxxxxxxxxx>
```
We will need the `Image URI` for our SageMaker pipeline setup. You can also find this image URI from the [ECR Console](https://console.aws.amazon.com/ecr/repositories) (make sure the correct region is selected in the ECR console).

Initialize a variable with the Docker image URI that we built in the previous step, we will also store this variable in cache

In [ ]:
# Copy the Image URI from the log output below
#------------------------------------------------

# CONTAINER_IMAGE_URI= "<ACCOUNT_ID>.dkr.ecr.<REGION>.amazonaws.com/sagemaker-studio-d-xxxxxxxxx:default-<xxxxxxxxxxx>"

%store CONTAINER_IMAGE_URI


# 3. Conclusion
---

In this notebook, we

* Installed the sagemaker-studio-image-build CLI tool that helps us bulding and publishing custom docker images for our ML Workstream
* Setup IAM permissions for the CLI tool
* Built the docker image using the `sm-docker build` command that includes Boto3 and SageMaker SDK libraries
* Initialized the variable `CONTAINER_IMAGE_URI` with the resulting Image URI and stored it in cache for use later